In [1]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import vstack, hstack, join, unique
from astroquery.simbad import Simbad
from astropy.time import Time

import sys
sys.path.append('../')
from uvot import read_uvotsource, calc_uvot_flags

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
for path in uvotsource_files:
    tab = read_uvotsource(path)
    diff = np.diff(tab['MJD'])
    #lo = np.mean(diff)-np.std(diff)
    hi = np.mean(diff)+np.std(diff)

    plt.figure(figsize=(12,6))
    plt.title(path)
    plt.plot(diff)
    plt.axhspan(0, hi, alpha=0.5, color='cyan', ec='black')
    plt.show()

NameError: name 'uvotsource_files' is not defined

In [ ]:

def plot_uvotsource_mag_hist(path, ax=None):
    tab = read_uvotsource(path)
    
    if ax is None:
        fig, ax = plt.subplots()
    ax.hist(tab['MAG'], bins=20, orientation="horizontal")

In [ ]:
tab = read_uvotsource(path)

In [ ]:
fig = plt.figure(figsize=(20,20))
ax1 = plt.subplot2grid((7,7), (0,0), colspan=6) # Lightcurve
ax2 = plt.subplot2grid((7,7), (0,6))
ax3 = plt.subplot2grid((7,7), (1,0), colspan=6)
ax3 = plt.subplot2grid((7,7), (2,0), colspan=6)
#ax4 = plt.subplot2grid((3,3), (2, 0))
#ax5 = plt.subplot2grid((3,3), (2, 1))

In [ ]:
import matplotlib.gridspec as gridspec

gs1 = gridspec.GridSpec(3, 3)
gs1.update(left=0.05, right=0.48, wspace=0.05)
ax1 = plt.subplot(gs1[:-1, :])
ax2 = plt.subplot(gs1[-1, :-1])
ax3 = plt.subplot(gs1[-1, -1])

gs2 = gridspec.GridSpec(3, 3)
gs2.update(left=0.55, right=0.98, hspace=0.05)
ax4 = plt.subplot(gs2[:, :-1])
ax5 = plt.subplot(gs2[:-1, -1])
ax6 = plt.subplot(gs2[-1, -1])

In [ ]:
fig, ax = plt.subplots()

In [ ]:
glob('../download_scripts/*/*uvot*.fits')

In [ ]:
tab = Table.read('../download_scripts/NGC7793/src_uvotsource_all.fits')

In [ ]:
tab['UPPER_LIM'] = tab['NSIGMA'] < tab['MAG_LIM_SIG'] 
tab = tab[tab['EXTNAME'] == 'U']
tab = tab[tab['MAG'] != 99]

In [ ]:
tab['EXTNAME', 'MAG', 'MAG_LIM', 'MAG_LIM_SIG', 'NSIGMA', 'UPPER_LIM']

In [ ]:
for t in tab.colnames:
    plt.figure(figsize=(3,3))
    plt.title(t)
    try:
        plt.hist(tab[t], bins=20)
    except:
        pass
    plt.show()

In [ ]:
plt.figure(figsize=(20,5))
#plt.scatter(tab['MET'], tab['MAG'])
#plt.scatter(tab['MET'], tab['MAG_LIM'])
plt.errorbar(tab['MET'], tab['MAG'], yerr=tab['MAG_ERR'], ls='none', uplims=tab['UPPER_LIM'], lw=1.0)
#plt.axhline(3)
#plt.ylim(18,22)

In [ ]:
tab = tab[tab['UPPER_LIM'] == False]

In [ ]:
plt.figure(figsize=(20,5))
#plt.scatter(tab['MET'], tab['MAG'])
#plt.scatter(tab['MET'], tab['MAG_LIM'])
plt.errorbar(tab['MET'], tab['MAG'], yerr=tab['MAG_ERR'], ls='none', uplims=tab['UPPER_LIM'], lw=1.0)
#plt.axhline(3)
#plt.ylim(18,22)

In [ ]:
from astropy.timeseries import LombScargle

In [ ]:
f, y = LombScargle(tab['MET'], tab['MAG']).autopower()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(f,y)

In [ ]:
uvotsource_lcs = glob('../download_scripts/*/*uvotsource_all.fits')

In [ ]:
for lc in uvotsource_lcs:
    tab = Table.read(lc, format='fits')
    tab = calc_uvot_flags(tab)
    tab = tab[~tab['FLAG_AB_MAG_99']]
    tab.sort('MET')
    
    
    fig, ax = plt.subplots(1, 1, figsize=(20,5))
    ax.set_title(lc)
    filter_colors = {'V' : 'red',
                     'B' : 'black',
                     'U' : 'blue',
                     'UVW1' : 'green',
                     'UVW2' : 'purple',
                     'UVM2' : 'cyan',
                     'WHITE' : 'grey'}
    
    for f in np.unique(tab['FILTER']):
        sub = tab[tab['FILTER'] == f]
        ax.scatter(sub['MET'], sub['MAG'], label=f, color=filter_colors[f])
    ax.legend()
    plt.show()